# 1. Load data

In [336]:
import pandas as pd
import ast
from utils.label import *
import re
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [337]:
df = pd.read_csv("./5000_merged/res.csv")
df =  df.drop(df.columns[[0, 1]],axis = 1)

In [338]:
print(df)

                                                     q  \
0              如何識別潛在的高風險高回報ETFs，並評估其與傳統低風險ETF之間的平衡策略？   
1                     ETF的流動性如何影響我作為一名散戶投資者進入和退出市場的時機？   
2    那些因素可能導致ETF的折溢價，即其交易價格與其淨資產價值(NAV)不一致，且我應如何運用這...   
3                       在選擇行業特定ETF時，應如何分析其成分股的多樣性和集中度？   
4             投資於主動管理式ETF與被動指數追蹤ETF的性能比較如何，考慮到不同的市場環境？   
..                                                 ...   
995    從技術分析的角度來看，<companyname>目前是否位於重要的支撐或阻力水平？（個股分析）   
996  <companyname>過去一年的營運活動現金流有哪些主要變化？這對股價有何影響？（個股分...   
997     根據<companyname>的行業地位和市場占有率，能否推斷它未來的業績增長？（個股分析）   
998      鑒於<companyname>的債務管理策略，是否應對其長期信貸風險保持警惕？（個股分析）   
999  如果想獲得穩定的股利回報，<companyname>的股利發放記錄和未來預期如何？（個股分析...   

                                           y  
0    [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1]  
1    [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1]  
2    [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1]  
3    [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1]  
4    [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1]  
..                   

In [339]:
tmp = df['y'].apply(ast.literal_eval)
df_dummy = pd.DataFrame(tmp.tolist(), columns=[f'col_{i}' for i in range(13)])

In [340]:
df = pd.concat([df, df_dummy], axis=1)

In [341]:
label_map = {
    '個股分析':0,
    '多股比較':1,	
    '個股資料查詢':2,	
    '產業':3,
    '大盤':4,
    '實務':5,
    '推薦軟條件':6,	
    '選股硬條件':7,	
    '字典查詢':8,
    '預測':9,	
    '不處理':10,	
    '總經與其他金融工具':11,
    '學習請教':12,
}

In [342]:
df = df.rename(columns={f'col_{i}': f'{col}' for i, col in enumerate(label_map.keys())})

# 2. clean data

## 2.1 修正多股比較_標籤
含有```'<othercompanyname>'```的一律視為有多股比較

In [343]:
df.head()

,q,y,個股分析,多股比較,個股資料查詢,產業,大盤,實務,推薦軟條件,選股硬條件,字典查詢,預測,不處理,總經與其他金融工具,學習請教
0,如何識別潛在的高風險高回報ETFs，並評估其與傳統低風險ETF之間的平衡策略？,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1]",0,0,0,0,1,0,0,0,0,1,0,1,1
1,ETF的流動性如何影響我作為一名散戶投資者進入和退出市場的時機？,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1]",0,0,0,0,1,0,0,0,0,1,0,1,1
2,那些因素可能導致ETF的折溢價，即其交易價格與其淨資產價值(NAV)不一致，且我應如何運用這...,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1]",0,0,0,0,1,0,0,0,0,1,0,1,1
3,在選擇行業特定ETF時，應如何分析其成分股的多樣性和集中度？,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1]",0,0,0,0,1,0,0,0,0,1,0,1,1
4,投資於主動管理式ETF與被動指數追蹤ETF的性能比較如何，考慮到不同的市場環境？,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1]",0,0,0,0,1,0,0,0,0,1,0,1,1


In [344]:
substring_to_check = '<othercompanyname>'
mask = df['q'].str.contains(substring_to_check, case=True)
df.loc[mask, '多股比較'] = 1

In [345]:
df.head()

,q,y,個股分析,多股比較,個股資料查詢,產業,大盤,實務,推薦軟條件,選股硬條件,字典查詢,預測,不處理,總經與其他金融工具,學習請教
0,如何識別潛在的高風險高回報ETFs，並評估其與傳統低風險ETF之間的平衡策略？,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1]",0,0,0,0,1,0,0,0,0,1,0,1,1
1,ETF的流動性如何影響我作為一名散戶投資者進入和退出市場的時機？,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1]",0,0,0,0,1,0,0,0,0,1,0,1,1
2,那些因素可能導致ETF的折溢價，即其交易價格與其淨資產價值(NAV)不一致，且我應如何運用這...,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1]",0,0,0,0,1,0,0,0,0,1,0,1,1
3,在選擇行業特定ETF時，應如何分析其成分股的多樣性和集中度？,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1]",0,0,0,0,1,0,0,0,0,1,0,1,1
4,投資於主動管理式ETF與被動指數追蹤ETF的性能比較如何，考慮到不同的市場環境？,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1]",0,0,0,0,1,0,0,0,0,1,0,1,1


## 2.2 刪除 q 後面註記的()

In [346]:
def _del(s):
    pattern  = re.compile(r'？\（.*\）')
    return pattern.sub("？", s)

In [347]:
df['q'] = df['q'].apply(_del)

## 2.3 <>裡面的東西一律改成:companyname

In [348]:
def _cmpyname(s):
    pattern  = re.compile(r'<[^>]+>')
    return pattern.sub("<companyname>", s)

In [349]:
df['q'] = df['q'].apply(_cmpyname)

## 2.4 轉成繁體

In [350]:
from tqdm import tqdm
tqdm.pandas()

In [351]:
import opencc
converter= opencc.OpenCC('s2twp.json')
def simplify_to_traditional(simplified_text, converter = converter):
    traditional_text = converter.convert(simplified_text)
    return traditional_text

In [352]:
df['q'] = df['q'].apply(simplify_to_traditional)

## 2.5 個股分析標籤新增
規則:包含```<companyname>```但是不包含```<othercompanyname>```

In [353]:
mask1 = df['q'].str.contains('<companyname>')
mask2 = df['q'].str.contains('<othercompanyname>')
mask = ((mask1) != (mask1 & mask2))

In [354]:
df.loc[mask, '個股分析'] = 1

# 3. 匯出

In [355]:
df = df.drop(df.columns[1], axis=1)

In [356]:
df = df.sample(frac=1).reset_index(drop=True)

In [357]:
df.to_csv('./5000_final/cleanres.csv', encoding='utf-8-sig')

In [358]:
df.shape

(1000, 14)